## Applying Machine Learning To Sentiment Analysis

The IMDB movie review set can be downloaded from http://ai.stanford.edu/~amaas/data/sentiment/

In [ ]:
import pandas as pd
import numpy as np

#add proper file path here
df = pd.read_csv('...')

In [ ]:
df.shape

In [ ]:
# Prepare cleaning functions
import re, string
import nltk
from nltk.stem import SnowballStemmer

stop_words = ["a", "an", "the", "this", "that", "is", "it", "to", "and"]

stemmer = SnowballStemmer('english')

def preProcessText(text):
    # lowercase and strip leading/trailing white space
    text = text.lower().strip()
    
    # remove HTML tags
    text = re.compile('<.*?>').sub('', text)
    
    # remove punctuation
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    
    # remove extra white space
    text = re.sub('\s+', ' ', text)
    
    return text

def lexiconProcess(text, stop_words, stemmer):
    filtered_sentence = []
    words = text.split(" ")
    for w in words:
        if w not in stop_words:
            filtered_sentence.append(stemmer.stem(w))
    text = " ".join(filtered_sentence)
    
    return text

def cleanSentence(text, stop_words, stemmer):
    return lexiconProcess(preProcessText(text), stop_words, stemmer)

In [ ]:
# This is the first review
df['review'].values[0]

In [ ]:
# Clean Up Approach 1: Call the cleanSentence() function on each item from the review column
# df['review'] = [cleanSentence(item, stop_words, stemmer) for item in df['review'].values]

In [ ]:
# Clean Up Apprach 2: Apply the cleanSentence() function as a lambda function directly on the review column with apply()
df['review'] = df['review'].apply(lambda x : cleanSentence(x, stop_words, stemmer))

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Or you can use a tf-idf vectorizer 
# tfidf = TfidfVectorizer()

textvectorizer = CountVectorizer(binary=True)
tokens = textvectorizer.fit_transform(df['review'])
tokens.shape

In [ ]:
# Prepare the data: use a much smaller set (!) for training and last 10000 for testing. 
X_train = tokens[:10000]
y_train = df.iloc[:10000, 1]
# X_train = tokens[:40000]
# y_train = df.iloc[:40000, 1]

X_test = tokens[40000:]
y_test = df.iloc[40000:, 1]

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)
print(classification_report(y_test, y_predict))

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True)),
    ('dt', DecisionTreeClassifier())                           
])

In [ ]:
# And use the train_test_split to prepare the train test sets
# The df dataframe has been cleaned. 
# Using a much smaller dataset (!) 

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df[:1000], test_size=0.1, shuffle=True, random_state=42)

X_train = train_data['review']
y_train = train_data['sentiment']
X_test = test_data['review']
y_test = test_data['sentiment']


In [ ]:
# Fit the pipeline 
pipeline.fit(X_train, y_train)

In [ ]:
# Evaluate the pipeline on the test set
from sklearn.metrics import classification_report
y_predict = pipeline.predict(X_test)
print(classification_report(y_test, y_predict))

In [ ]:
label = {0: 'negative', 1: 'positive'}
example = ['The movie was not worthy of my time, and I was very disappointed. The story was so dull and the characters were lame. ']

print('Prediction:', label[pipeline.predict(example)[0]])

In [ ]:
import pickle

with open('model.pkl', 'wb') as model_file:
  pickle.dump(pipeline, model_file)

In [ ]:
saved_model_pipeline = pickle.load(open('model.pkl', "rb" ))

- Task 1: load the saved model from file and create a new model object. 
- Task 2: predict a list of reviews using the new model object. 